Importieren des Dataframes:

In [ ]:
import numpy as np
import pandas as pd
import folium as fo
from folium.plugins import HeatMap

NextBike = pd.read_csv("touren_Nextbike.csv")
NextBike.head()
print()


,Unnamed: 0,FahrradID,AusleihstationID,AusleihstationName,AusleihstationPLZ,AusleihzeitID,RueckgabestationID,RueckgabestationName,RueckgabestationPLZ,Rueckgabezeit,Dauer,Rueckgabe_datetime,Rueckgabe_Stunde,Rueckgabe_Stunde_des_Tages,start_lat,start_lon,end_lat,end_lon,dist_km
0,0,42730,-1,unbekannt,60000,120145,339475,Hauptbahnhof Mannheim,68001.0,120146,1,2025-05-28 22:10:00,2025-05-28 22:00:00,22,49.479960,8.469749,49.479960,8.469749,0.000000
1,1,42730,339475,Hauptbahnhof Mannheim,68001,120195,339343,Neckarstadt - Alter Messplatz,68169.0,120206,11,2025-05-28 23:10:00,2025-05-28 23:00:00,23,49.479960,8.469749,49.497070,8.473044,1.917374
2,2,42730,339343,Neckarstadt - Alter Messplatz,68169,120235,339466,G7 - Westliche Unterstadt,68159.0,120246,11,2025-05-28 23:50:00,2025-05-28 23:00:00,23,49.497070,8.473044,49.491183,8.462391,1.010192
3,3,42730,339466,G7 - Westliche Unterstadt,68159,120725,148606240,A1 - Landgericht/Universität,68002.0,120736,11,2025-05-29 08:00:00,2025-05-29 08:00:00,8,49.491183,8.462391,49.484796,8.462552,0.710297
4,4,42730,148606240,A1 - Landgericht/Universität,68002,120745,339472,A5 - Universität West,68002.0,120746,1,2025-05-29 08:10:00,2025-05-29 08:00:00,8,49.484796,8.462552,49.487110,8.456876,0.484062


Einfügen der Spalte "No_Move" mit Wert True falls das Fahrrad nicht bewegt wurde und False, falls es eine Bewegung gab

In [2]:
NextBike["No_Move"]=(NextBike["start_lat"]==NextBike["end_lat"])&(NextBike["start_lon"]==NextBike["end_lon"]) 
#NextBike.head()

Erstellen des Dataframes, welcher nur Bewegungen enthält.

In [3]:
Clean_NextBike = NextBike[NextBike["No_Move"]==False]
AnzahlAusleihenStation = Clean_NextBike["AusleihstationID"].value_counts()
AnzahlRueckgabenStation = Clean_NextBike["RueckgabestationID"].value_counts()
Clean_NextBike.head()
print(Clean_NextBike.shape)

(359806, 20)


Erstellen einer Liste der einzelnen Stationen die es gibt

In [4]:
Stationen = Clean_NextBike.drop_duplicates(subset="AusleihstationID")
Stationen.drop(columns=["Unnamed: 0","Dauer","FahrradID","AusleihzeitID","RueckgabestationID","RueckgabestationName","RueckgabestationPLZ","Rueckgabezeit","Rueckgabe_datetime","Rueckgabe_Stunde","Rueckgabe_Stunde_des_Tages","end_lat","end_lon","dist_km","No_Move"],inplace=True)

for AusleihstationID_IT in Stationen["AusleihstationID"]:
    Stationindex = Stationen.index[Stationen["AusleihstationID"]==AusleihstationID_IT]
    Stationen.loc[Stationindex,"Ausleihcount"] = AnzahlAusleihenStation.get(AusleihstationID_IT)
    Stationen.loc[Stationindex,"Rueckgabecount"] = AnzahlRueckgabenStation.get(AusleihstationID_IT)
Stationen.head(20)

,AusleihstationID,AusleihstationName,AusleihstationPLZ,start_lat,start_lon,Ausleihcount,Rueckgabecount
1,339475,Hauptbahnhof Mannheim,68001,49.479960,8.469749,25156.0,24845.0
2,339343,Neckarstadt - Alter Messplatz,68169,49.497070,8.473044,7840.0,7841.0
3,339466,G7 - Westliche Unterstadt,68159,49.491183,8.462391,5758.0,5763.0
4,148606240,A1 - Landgericht/Universität,68002,49.484796,8.462552,8475.0,8481.0
5,339472,A5 - Universität West,68002,49.487110,8.456876,15776.0,15797.0
6,339354,K1 - Kurpfalzbrücke,68169,49.493002,8.470325,7580.0,7578.0
8,58196617,Lindenhof / NTR,68003,49.476208,8.470888,1832.0,1835.0
9,30612264,B3 – Schillerplatz,68159,49.485946,8.462150,5229.0,5240.0
10,30612203,Jungbusch - Beilstraße,68159,49.494500,8.459644,4421.0,4425.0
11,27230871,Neckarstadt-Ost - Herzogenriedpark,68167,49.503921,8.477009,2987.0,3005.0


Erstellen von ersten Statistiken
avgdistanz = durchschnittliche Distanz
AnzahlAusleihen = Anzahl der Ausleihen als Zählung der Häufigkeit der StationsID als AusleihstationsID
AnzahlRueckgaben = Anzahl der Rückgaben als Zählung der Häufigkeit der StationsID als RueckgabestationsID
AnzahlAusleihenRueckgaben = Anzahl der Strecken zwischen einer Station als Start und einer als Endstation als Count des Paares (AusleihstationsID,RueckgabestationsID)

In [5]:
avgdistanz = Clean_NextBike["dist_km"].mean()
AnzahlAusleihen = Clean_NextBike["AusleihstationID"].value_counts()
AnzahlRueckgaben = Clean_NextBike["RueckgabestationID"].value_counts()
AnzahlAusleihenRueckgaben = Clean_NextBike.value_counts(subset=["AusleihstationID","RueckgabestationID"])
print(f"Die Durchschnittliche Fahrdistanz sind {avgdistanz} Kilometer\n")

Die Durchschnittliche Fahrdistanz sind 1.5110902394376327 Kilometer



In [6]:
AnzahlAusleihenPrint= "\n".join(f" {item}: {count}" for item, count in AnzahlAusleihen.items()) 
#print(AnzahlAusleihenPrint)

AnzahlRueckgabenPrint= "\n".join(f" {item}: {count}" for item, count in AnzahlRueckgaben.items()) 
#print(AnzahlRueckgabenPrint)

AnzahlAusleihenRueckgabenPrint= "\n".join(f" {item}: {count}" for item, count in AnzahlAusleihenRueckgaben.items()) 
#print(AnzahlAusleihenRueckgabenPrint)

Vorbereiten der Marker für die Nextbike Stationen auf der Karte (Die Stationen ohne Längen/Breitengrad müssen entfernt werden)

In [7]:
StationMarker = Stationen.loc[:,["start_lat","start_lon","AusleihstationName","Ausleihcount","Rueckgabecount"]]
StationMarker.dropna(inplace=True)
StationMarker

,start_lat,start_lon,AusleihstationName,Ausleihcount,Rueckgabecount
1,49.479960,8.469749,Hauptbahnhof Mannheim,25156.0,24845.0
2,49.497070,8.473044,Neckarstadt - Alter Messplatz,7840.0,7841.0
3,49.491183,8.462391,G7 - Westliche Unterstadt,5758.0,5763.0
4,49.484796,8.462552,A1 - Landgericht/Universität,8475.0,8481.0
5,49.487110,8.456876,A5 - Universität West,15776.0,15797.0
...,...,...,...,...,...
2297,49.467531,8.503804,Neuhermsheim - Dragonerweg,268.0,276.0
2896,49.525003,8.481472,S-Bahnhof Waldhof,768.0,764.0
3314,49.525543,8.535079,Franklin - George-Sullivan-Ring,259.0,256.0
4747,49.521503,8.525401,Franklin - Wasserwerkstraße,462.0,459.0


Erstellen der Karte mit den einzelnen Stationen mit interaktiver Namensanzeige der Station

In [8]:
Kartefig=fo.Figure(width=1000, height=500)
KarteStationen = fo.Map([49.487956,8.466007],zoom_start=15).add_to(Kartefig)
for i in range(0,len(StationMarker)):
    fo.Marker(location=[StationMarker.iloc[i]["start_lat"],StationMarker.iloc[i]["start_lon"]], popup=StationMarker.iloc[i][["AusleihstationName","Ausleihcount","Rueckgabecount"]]).add_to(KarteStationen)
KarteStationen.show_in_browser()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.


Erstellen der (Luftlinien)Routen für die gefahrenen Nextbike Strecken und einfügen in die neue Karte

In [ ]:
LinePoints_coordinates= Clean_NextBike.loc[:,["start_lat","start_lon","end_lat","end_lon"]]
LinePoints_coordinates.dropna(inplace=True)
#Erstellen der neuen Karte basierend auf der alten Karte
KarteLinien = KarteStationen
for i in range(0,len(LinePoints_coordinates)):
    LineCoordinates = [[LinePoints_coordinates.iloc[i]["start_lat"],LinePoints_coordinates.iloc[i]["start_lon"]] , [LinePoints_coordinates.iloc[i]["end_lat"],LinePoints_coordinates.iloc[i]["end_lon"]]]
    Line = fo.PolyLine(locations=LineCoordinates, color = "green")
    Line.add_to(KarteLinien)
KarteLinien.show_in_browser()    

https://project-osrm.org/docs/v5.5.1/api/#general-options

Link zu Doku wie mann aus Open-Street Map routen-Daten extrahieren kann

